## Imports

In [104]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import requests
from bs4 import BeautifulSoup

## Read-in

In [105]:
df = pd.read_csv('CookCountyFeb-Mar Foreclosures - feb_mar_foreclosures.csv')

In [106]:
df = df.drop(columns=['Color','COLOR'])

## Clean, drop, and convert

In [107]:
df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'])

## Data Stuff

In [108]:
# Count number of February dates
feb_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 2])

print(f'Number of February dates: {feb_dates_count}')

Number of February dates: 662


In [109]:
# Count number of February dates
mar_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 3])

print(f'Number of March dates: {mar_dates_count}')

Number of March dates: 778


In [110]:
df = df.drop(columns='1st PIN')

In [111]:
df['1st Grantor'] = df['1st Grantor'].fillna('NA')
df['1st Grantee'] = df['1st Grantee'].fillna('NA')

In [112]:
municipal_authority = ['city', 'town', 'municipality', 'village','transit auth','department of transp']  # list of municipal authority keywords
df = df[~df['1st Grantor'].str.contains('|'.join(municipal_authority), case=False) & 
        ~df['1st Grantee'].str.contains('|'.join(municipal_authority), case=False)]

In [113]:
# create new column and assign colors based on month
df['COLOR'] = df['Doc Recorded'].apply(lambda x: 'orange' if x.month == 2 else 'red' if x.month == 3 else '')

## Get mortgage amounts

In [114]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
}

In [115]:
def mortgage_url_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/Document/Detail'):
            mortgage_url = 'https://crs.cookcountyclerkil.gov' + link['href']
            return mortgage_url

In [116]:
def mortgage_consi_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all tr tags
    trs = soup.find_all('tr')

    # Loop through each tr tag and look for the td tag containing 'Consideration Amount' label
    for tr in trs:
        td = tr.find('th', text='Consideration Amount:')
        if td:
            # If the td tag is found, get the next td tag containing the amount
            amount_td = td.find_next_sibling('td')
            if amount_td:
                # Print the amount
                amount = amount_td.text.strip()
                return amount
            else:
                return 'not found'

In [117]:
df['mortgage_urls'] = df['deed_urls'].apply(lambda x: mortgage_url_snagger(x, headers))

In [118]:
df['mortgage_amount'] = df['mortgage_urls'].apply(lambda x: mortgage_consi_snagger(x, headers) if x is not None else None)

In [119]:
df[['mortgage_urls','mortgage_amount']] = df[['mortgage_urls','mortgage_amount']].fillna('NA')

## HTML Popup Formatter

In [121]:
df.columns

Index(['Doc Number', 'Doc Recorded', 'Doc Executed', 'Doc Type', 'Consi. Amt.',
       '1st Grantor', '1st Grantee', 'Assoc. Doc#', 'deed_urls',
       'folio_number', 'address', 'geo_address', 'geocoded', 'lat', 'lon',
       'COLOR', 'mortgage_urls', 'mortgage_amount'],
      dtype='object')

In [122]:
def popup_html(row):
    grantor = row['1st Grantor']
    grantee = row['1st Grantee']
    folio = row['folio_number']
    address = row['address']
    mortgage_amount = row['mortgage_amount']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Lendor: </strong>{}'''.format(grantor) + '''<br>
    <strong>Lendee: </strong>{}'''.format(grantee) + '''<br>
    <strong>PIN: </strong>{}'''.format(folio) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Mortgage Amount: </strong>{}'''.format(mortgage_amount) + '''<br>
    </html>
    '''
    return html

In [123]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Cook County Foreclosures')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February & March')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_orange = folium.FeatureGroup(name='February')
fg_red = folium.FeatureGroup(name='March')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'orange':
        marker = folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            color='orange',
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)
    else:
        marker = folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            color=row['COLOR'],
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_red)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_red.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
folium.TileLayer('CartoDBpositron',control=False).add_to(m) 
            
# Display map
m

In [124]:
m.save('index.html')

## Map URL Snagger

In [125]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/lis_pendens_scraper


## Stats for story

In [126]:
# Count number of February dates
feb_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 2])

print(f'Number of February dates: {feb_dates_count}')

# Count number of February dates
mar_dates_count = len(df.loc[df['Doc Recorded'].dt.month == 3])

print(f'Number of March dates: {mar_dates_count}')

Number of February dates: 662
Number of March dates: 770


In [127]:
feb = df.loc[df['Doc Recorded'].dt.month == 2]
mar = df.loc[df['Doc Recorded'].dt.month == 3]

In [128]:
feb['1st Grantor'].value_counts().head(60)

NATIONSTAR MTG LLC                    40
US BK NATL ASSN                       39
US BK NATL ASSN TR                    37
LAKEVIEW LOAN SERVICING LLC           35
NEWREZ LLC                            32
US BK TRUST NATL ASSN TR              28
WILMINGTON SAV FUND SOC FSB TR        26
WELLS FARGO BK NA                     21
FREEDOM MTG CORP                      18
PENNYMAC LOAN SERVICES LLC            17
CARRINGTON MTG SERVICES LLC           16
MIDFIRST BK                           14
DEUTSCHE BK NATL TRUST CO TR          13
LOANDEPOTCOM LLC                      11
JPMORGAN CHASE BK NATL ASSN           10
PNC BK NATL ASSN                       9
THE BK OF NEW YORK MELLON TR           9
ROCKET MTG LLC                         7
FEDERAL HOME LOAN MTG CORP TR          7
FIFTH THIRD BK NATL ASSN               7
UNITED WHOLESALE MTG LLC               6
CITIZENS BK NA                         5
GOLBERT CHARLES P GDN                  5
MORTGAGE ASSETS MGMT LLC               5
SELECT PORTFOLIO

In [129]:
feb['1st Grantee'].value_counts()

UNKNOWN                           8
HAB GRP LLC                       4
CHICAGO TITLE LAND TRUST CO TR    4
FIFTH THIRD BK                    2
KALBARCZYK JADWIGA                2
                                 ..
CARSON ESTELLA M                  1
MILLER CYNTHIA A                  1
HOROPCIUC BOGDAN                  1
MCSWAIN JOHN LOUIS SR             1
SALGADO ISIDRO                    1
Name: 1st Grantee, Length: 638, dtype: int64

In [130]:
mar['1st Grantor'].value_counts()

US BK NATL ASSN                    56
LAKEVIEW LOAN SERVICING LLC        46
US BK NATL ASSN TR                 42
NATIONSTAR MTG LLC                 39
US BK TRUST NATL ASSN TR           36
                                   ..
ANTLER MTG LOAN TRUST 2021-RTL1     1
FORETHOUGHT LIFE INST CO            1
MIDLAND STATES BK                   1
RELIANCE FIRST CAP LLC              1
U S BK NATL ASSN                    1
Name: 1st Grantor, Length: 194, dtype: int64

In [131]:
mar['1st Grantee'].value_counts()

CHICAGO TITLE LAND TRUST CO TR    5
UNKNOWN                           4
SMITH JAMES                       3
DELEE HOLDINGS LLC                3
JEBB FOOD SERVICES INC            2
                                 ..
CHANDLER DIONISIO                 1
WINES DOROTHY                     1
SALGADO PEDRO                     1
MCBRIDE MONIQUE D                 1
MARTIN ELIZABETH                  1
Name: 1st Grantee, Length: 753, dtype: int64

In [132]:
df.to_csv('feb_mar_foreclosures.csv')